#SentimentAnalyzer : 

* ## 1

----

Import the “Subjectivity” corpus using from nltk.corpus import subjectivity

----

In [ ]:
from nltk.corpus import subjectivity
import nltk
nltk.download('subjectivity')

[nltk_data] Downloading package subjectivity to /root/nltk_data...
[nltk_data]   Unzipping corpora/subjectivity.zip.


True

* ## 2

---
The dataset contains two categories of instances: ’subj’ and ’obj’. let's create a training set and a test set containing, respectively, 320 and 80 instances of each category using subjectivity.sents (categories = ...).


---

In [ ]:
train_instances = 320
test_instances = 80
n_inst = 400
# Each document is represented by a tuple (sentence, label).
# The sentence is tokenized, so it is represented by a list of strings:
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_inst]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_inst]]

In [ ]:
sliceTrain = int(train_instances/2)
sliceTest = int(test_instances/2)
train_subj_docs = subj_docs[:sliceTrain]
test_subj_docs = subj_docs[sliceTrain:sliceTrain+sliceTest]
train_obj_docs = obj_docs[:sliceTrain]
test_obj_docs = obj_docs[sliceTrain:sliceTrain+sliceTest]
training_docs = train_subj_docs+train_obj_docs
testing_docs = test_subj_docs+test_obj_docs

In [ ]:
len(testing_docs)

80

In [ ]:
len(training_docs)

320

* ## 3

---

Use the mark_negation function to highlight negations in the data.

---

In [ ]:
from nltk.sentiment.util import *

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
neg = [mark_negation(training_docs[i][0]) for i in range(len(training_docs))]

In [ ]:
neg[2]

['it',
 'is',
 'not',
 'a_NEG',
 'mass-market_NEG',
 'entertainment_NEG',
 'but_NEG',
 'an_NEG',
 'uncompromising_NEG',
 'attempt_NEG',
 'by_NEG',
 'one_NEG',
 'artist_NEG',
 'to_NEG',
 'think_NEG',
 'about_NEG',
 'another_NEG',
 '.']

* ## 4

---

Define an unigram extractor, named uni_feat_extractor, using the unigram_word_feats function of SentimentAnalyzer. Keep only un-igrams that have a minimum frequency equal to 5 (min_freq = 5)

---

In [ ]:
from nltk.sentiment import SentimentAnalyzer
sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])
uni_feat_extractor = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=5)

In [ ]:
print(uni_feat_extractor)

['.', 'the', ',', 'a', 'and', 'of', 'to', 'is', 'in', 'that', 'his', 'for', 'with', 'it', 'as', 'an', 'on', 'who', 'he', 'by', 'this', 'film', 'has', 'movie', 'from', 'but', 'her', 'are', 'their', 'the_NEG', '"', 'its', 'one', 'at', '(', 'be', ',_NEG', ')', 'when', 'they', 'a_NEG', 'about', 'not', 'to_NEG', 'into', 'all', 'out', 'so', 'will', 'him', 'life', 'have', 'most', 'of_NEG', 'she', 'story', 'you', 'even', 'like', ';', 'more', "it's", 'if', ':', 'what', 'only', "there's", 'can', 'love', 'where', 'and_NEG', 'which', 'up', 'take', 'home', 'while', 'both', 'make', 'some', 'work', 'in_NEG', 'get', 'through', 'gets', 'i', '--', 'two', 'or', 'old', 'come', 'beautiful', 'kids', 'just', 'but_NEG', 'much', 'look', 'before', 'it_NEG', 'great', 'very', 'really', 'own', "doesn't", 'no', 'young', 'funny', 'far', 'world', 'than', 'makes', 'years', 'way', 'may', 'them', 'new', 'between', 'search', 'family', 'daughter', 'been', 'time', 'never', 'is_NEG', 'after', 'feel', 'begins', 'how', 'peopl

* ## 5

---

Add the extractor to the previously defined SentimentAnalyzer () object, using the add_feat_extractor function.

---

In [ ]:
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=uni_feat_extractor)

* ## 6

---

Transform training and testing sets using apply_features function

---

In [ ]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)

* ## 7

---

Build two classifiers: Naive Bayes and SVM.

---

In [ ]:
from nltk.classify import NaiveBayesClassifier
import nltk.classify
from sklearn.svm import LinearSVC

In [ ]:
#NaiveBayesClassifier
trainer = NaiveBayesClassifier.train
NVclassifier = sentim_analyzer.train(trainer, training_set)
#SVM 
SVMclassifier = nltk.classify.SklearnClassifier(LinearSVC())
SVMclassifier.train(training_set)

Training classifier


<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))>

* ## 8

----

Test the classifiers obtained on the test set and display the evaluation metrics
using sentim_analyzer.evaluate (test_set)

-----

For naive bayses classifier

In [ ]:
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
   print('{0}: {1}'.format(key, value))

Evaluating NaiveBayesClassifier results...
Accuracy: 0.75
F-measure [obj]: 0.7435897435897436
F-measure [subj]: 0.7560975609756098
Precision [obj]: 0.7631578947368421
Precision [subj]: 0.7380952380952381
Recall [obj]: 0.725
Recall [subj]: 0.775


For SVM

In [ ]:
# get the accuracy using svm
accuracy = nltk.classify.accuracy(SVMclassifier, test_set)
print(accuracy)

0.7375


----
----
The best model is NaiveBayesClassifier ( it has the best accuracy )

----
----

* ## 9

----

Using the best model, we gonna predict the subjectivity of each document for the Brown corpus. Don't forget to transform the data using apply_features. And let's store the result in a Test_on_brown dictionary whose key is the document identifier (first document ID = 1) and the value indicates the subjectivity of the document ("obj" or "subj").


----

In [ ]:
from nltk.corpus import brown
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

Preparing our corpus data 

In [ ]:
brown_data = []
 
for fileid in brown.fileids():
    document = ' '.join(brown.words(fileid))
    brown_data.append(document)

----

In [ ]:
brown_set = sentim_analyzer.apply_features(brown_data)

In [ ]:
Test_on_brown={i+1:NVclassifier.classify(brown_set[i]) for i in range(0,len(brown_set))}

* ## 10



----
Let's the predicted subjectivity for document number 403 

----

In [ ]:
Test_on_brown[403]

'obj'

# Lexicon based Sentiment Analysis - Application à la langue Arabe

----

Develop a Senti_Analyzer_Ar function which takes as input a lexicon and an item in Arabic and returns the polarity of the item. The Lexicon to use is: Arabic Hashtag Lexicon (dialect), download from http://saifmohammad.com/WebPages/ArabicSA.html.

----

----

Downloading the arabic lexicon

----

In [ ]:
!wget http://saifmohammad.com/WebDocs/Arabic%20Lexicons/Arabic_Hashtag_Lexicon_dialectal.txt

--2021-01-12 21:15:09--  http://saifmohammad.com/WebDocs/Arabic%20Lexicons/Arabic_Hashtag_Lexicon_dialectal.txt
Resolving saifmohammad.com (saifmohammad.com)... 192.185.17.122
Connecting to saifmohammad.com (saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 616725 (602K) [text/plain]
Saving to: ‘Arabic_Hashtag_Lexicon_dialectal.txt’

Arabic_Hashtag_Lexi 100%[===================>] 602.27K  2.05MB/s    in 0.3s    

2021-01-12 21:15:10 (2.05 MB/s) - ‘Arabic_Hashtag_Lexicon_dialectal.txt’ saved [616725/616725]



----

We have to delete the information in the top of the file before uploading it 

In [ ]:
import numpy as np
import pandas as pd
 
lexicon = pd.read_csv('/content/Arabic_Hashtag_Lexicon_dialectal.txt', delimiter="\t", header=0)

In [ ]:
lexicon.tail()

,[Arabic Term],[Buckwalter],[Sentiment Score],[Positive Occurrence Count],[Negative Occurrence Count]
20122,العولقي,AlEwlqy,1.219,12,0.0
20123,مغلق,mglq,0.446,23,3.0
20124,جبلي,jbly,1.487,16,0.0
20125,نورة_الخطيب,nwrp_AlxTyb,0.600,6,0.0
20126,يوتيوب,ywtywb,1.793,322,13.0


----
----

In [ ]:
def Senti_Analyzer_Ar(lexicon, item):

  for i in range(len(lexicon['[Arabic Term]'])):
    if lexicon['[Arabic Term]'][i]== item:
      pola = lexicon['[Sentiment Score]'][i]
      break
    else: 
      pola = 0
  return pola

---

Test for the word 'اط'

---

In [ ]:
pola = Senti_Analyzer_Ar(lexicon, 'اط')
if pola  > -1:
            print('positive')
elif pola == 0:
            print('neutral')
elif pola  <= -1:
            print('negative')

positive


----

Let‘s use the Senti_Analyzer_Ar function to find the polarity of the items contained in the ”comments.csv” file

----

In [ ]:
comments = pd.read_excel('/content/comments.xlsx')

In [ ]:
comments.head()

,text,Unnamed: 1,Unnamed: 2
0,ههههههههه ما حدا احسن من حدا .. عشان تحسو فينا...,NaN,NaN
1,تحزنوا هاهاهاهاها,NaN,NaN
2,اللي بحكي انو الفيصلي عنده عقده اسمها شباب الأ...,NaN,NaN
3,يعني ادارة الفيصلي لما شافت فريقك الاول (الماج...,NaN,NaN
4,اوساسونا ريال سرقسطة,NaN,NaN


---

Putting the results in a dict 

----

In [ ]:
polarity_dict = {}
for i in range(len(comments['text'])):
  pola = 0
  for item in comments['text'][i].split():
    pola_i = Senti_Analyzer_Ar(lexicon, item)
    pola+=pola_i
    tmp = pola
  polarity_dict.__setitem__('comment %d polarity '% i , tmp)

----

Comment 2  polarity 

In [ ]:
polarity_dict['comment 2 polarity ']

1.306